# diSPIM Data Visualization Pipeline

This notebook provides a complete pipeline to load, process, and visualize high-resolution microscopy data from a double diSPIM (dual-view selective plane illumination microscopy) system.

## Overview

The data consists of:
- **Two imaging arms**: Alpha and Beta (each diSPIM)
- **Two channels per arm**: Two cameras/sides per diSPIM
- **Z-stacks**: 200 slices per volume
- **High resolution**: 2304×2304 pixels per slice
- **Format**: OME-TIFF files with detailed JSON metadata

## Goals

1. Load OME-TIFF image stacks from both alpha and beta arms
2. Extract and parse metadata for temporal and spatial alignment
3. Create side-by-side video visualizations
4. Enable interactive exploration of the data


## Section 1: Setup and Imports

First, we import all necessary libraries for data loading, processing, and visualization.


In [2]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from pathlib import Path
from datetime import datetime
import tifffile
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
import imageio
import warnings
warnings.filterwarnings('ignore')

# Set up matplotlib for inline display
# Use widget backend for interactive plots
%matplotlib notebook  
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['figure.dpi'] = 100

print("Libraries imported successfully!")


Libraries imported successfully!


## Section 2: Metadata Parser

The metadata files contain crucial information about the acquisition parameters. We need to parse the JSON structure to extract:
- Image dimensions (width, height, slices, channels)
- Temporal information (start time, slice period, volume duration)
- Spatial information (pixel size, z-step, position)
- Acquisition settings (delay between sides, etc.)

The metadata file has a nested structure with a "Summary" key containing most important parameters, and a "SPIMAcqSettings" field that contains a JSON string with additional settings.


In [3]:
def parse_metadata(metadata_path):
    """
    Parse the metadata JSON file and extract key acquisition parameters.
    
    Parameters:
    -----------
    metadata_path : str or Path
        Path to the metadata.txt file
        
    Returns:
    --------
    dict : Dictionary containing parsed metadata parameters
    """
    # Try different encodings to handle files that may not be UTF-8
    encodings = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
    metadata = None
    
    for encoding in encodings:
        try:
            with open(metadata_path, 'r', encoding=encoding, errors='replace') as f:
                metadata = json.load(f)
            break
        except (UnicodeDecodeError, json.JSONDecodeError):
            continue
    
    if metadata is None:
        # Last resort: try with errors='ignore' and latin-1 (can decode any byte)
        with open(metadata_path, 'r', encoding='latin-1', errors='ignore') as f:
            metadata = json.load(f)
    
    summary = metadata.get('Summary', {})
    
    # Parse the nested SPIMAcqSettings JSON string
    spim_settings_str = summary.get('SPIMAcqSettings', '{}')
    try:
        spim_settings = json.loads(spim_settings_str)
    except:
        spim_settings = {}
    
    # Extract key parameters
    parsed = {
        # Image dimensions
        'width': int(summary.get('Width', 0)),
        'height': int(summary.get('Height', 0)),
        'slices': int(summary.get('Slices', 0)),
        'channels': int(summary.get('Channels', 0)),
        'frames': int(summary.get('Frames', 1)),
        
        # Channel information
        'channel_names': summary.get('ChNames', []),
        'slices_first': summary.get('SlicesFirst', 'true').lower() == 'true',
        'time_first': summary.get('TimeFirst', 'false').lower() == 'true',
        
        # Spatial information
        'pixel_size_um': float(summary.get('PixelSize_um', 0)),
        'z_step_um': float(summary.get('z-step_um', 0)),
        'position_x': summary.get('Position_X', '0'),
        'position_y': summary.get('Position_Y', '0'),
        
        # Temporal information
        'start_time': summary.get('StartTime', ''),
        'slice_period_ms': float(summary.get('SlicePeriod_ms', '0 ms').split()[0]),
        'volume_duration_sec': float(summary.get('VolumeDuration', '0 s').split()[0]),
        
        # Acquisition settings
        'delay_before_side': spim_settings.get('delayBeforeSide', 0.25),
        'num_sides': spim_settings.get('numSides', 2),
        'first_side_is_a': spim_settings.get('firstSideIsA', True),
        
        # Other useful info
        'acquisition_name': summary.get('AcquisitionName', ''),
        'date': summary.get('Date', ''),
        'pixel_type': summary.get('PixelType', ''),
        'bit_depth': int(summary.get('BitDepth', 16)),
        'mv_rotations': summary.get('MVRotations', ''),
        
        # Full metadata for reference
        'raw_summary': summary,
        'raw_spim_settings': spim_settings
    }
    
    return parsed

# Test the parser
test_metadata_path = Path('1msec_worm/I/beads_alpha_worm2/beads_alpha_worm2_MMStack_Pos0_metadata.txt')
if test_metadata_path.exists():
    test_meta = parse_metadata(test_metadata_path)
    print("Metadata parser test successful!")
    print(f"Image dimensions: {test_meta['width']}x{test_meta['height']}x{test_meta['slices']}")
    print(f"Channels: {test_meta['channels']} ({', '.join(test_meta['channel_names'])})")
    print(f"Slice period: {test_meta['slice_period_ms']} ms")
    print(f"Volume duration: {test_meta['volume_duration_sec']} s")
else:
    print("Test metadata file not found. Parser function is ready to use.")


Metadata parser test successful!
Image dimensions: 2304x2304x200
Channels: 2 (HamCam2, HamCam1)
Slice period: 12.5 ms
Volume duration: 5.0255 s


## Section 3: OME-TIFF Loader

The OME-TIFF files contain multi-dimensional image stacks. Based on the metadata:
- Data organization: `SlicesFirst=True` means data is organized as [Slices, Channels, Height, Width]
- Each arm has 2 channels (two cameras/sides)
- 200 slices per volume
- 2304×2304 pixels per slice
- 16-bit grayscale data

We need to load the data and properly reshape it to separate channels and slices.


In [4]:
def load_ome_tiff(tiff_path, metadata=None, channel_idx=None, max_slices=None):
    """
    Load an OME-TIFF file and return properly shaped array.
    
    Parameters:
    -----------
    tiff_path : str or Path
        Path to the .ome.tif file
    metadata : dict, optional
        Parsed metadata dictionary. If provided, uses it to determine data organization.
        If None, will try to infer from file.
    channel_idx : int or None
        If specified, return only this channel (0-indexed). If None, return all channels.
    max_slices : int or None
        If specified, load only the first max_slices slices (for memory efficiency)
        
    Returns:
    --------
    numpy.ndarray : Image data
        Shape depends on parameters:
        - If channel_idx specified: (slices, height, width)
        - If channel_idx is None: (slices, channels, height, width) or (channels, slices, height, width)
    """
    print(f"Loading OME-TIFF: {tiff_path}")
    
    # Load the TIFF file
    # tifffile can handle OME-TIFF format and preserve metadata
    with tifffile.TiffFile(tiff_path) as tif:
        # Get the image series (OME-TIFF can have multiple series)
        if len(tif.series) > 0:
            # Get the first (and usually only) series
            series = tif.series[0]
            data = series.asarray()
        else:
            # Fallback: read directly
            data = tifffile.imread(tiff_path)
    
    print(f"Raw data shape: {data.shape}")
    print(f"Data dtype: {data.dtype}")
    
    # Determine data organization from metadata or infer from shape
    if metadata is not None:
        slices_first = metadata.get('slices_first', True)
        num_slices = metadata.get('slices', 200)
        num_channels = metadata.get('channels', 2)
    else:
        # Try to infer: if we have 4D data, assume [slices, channels, height, width]
        # or [channels, slices, height, width]
        if len(data.shape) == 4:
            # Check which dimension is larger (slices vs channels)
            if data.shape[0] > data.shape[1]:
                slices_first = True
                num_slices, num_channels = data.shape[0], data.shape[1]
            else:
                slices_first = False
                num_channels, num_slices = data.shape[0], data.shape[1]
        else:
            # 3D or 2D - assume it's already in the right format
            slices_first = True
            num_slices = data.shape[0] if len(data.shape) >= 3 else 1
            num_channels = 1
    
    # Limit slices if requested (for memory efficiency)
    if max_slices is not None and num_slices > max_slices:
        if slices_first:
            data = data[:max_slices]
        else:
            data = data[:, :max_slices]
        num_slices = max_slices
        print(f"Limited to {max_slices} slices")
    
    # Reshape if needed
    if len(data.shape) == 4:
        if slices_first:
            # Data is [slices, channels, height, width] - this is what we want
            pass
        else:
            # Data is [channels, slices, height, width] - transpose
            data = np.transpose(data, (1, 0, 2, 3))
    elif len(data.shape) == 3:
        # 3D data - need to determine if it's [slices, height, width] or [channels, height, width]
        if slices_first:
            # Assume it's [slices, height, width] with 1 channel
            data = data[:, np.newaxis, :, :]
        else:
            # Assume it's [channels, height, width] with 1 slice
            data = data[np.newaxis, :, :, :]
            data = np.transpose(data, (1, 0, 2, 3))
    elif len(data.shape) == 2:
        # Single 2D image
        data = data[np.newaxis, np.newaxis, :, :]
    
    # Extract specific channel if requested
    if channel_idx is not None:
        if len(data.shape) == 4:
            data = data[:, channel_idx, :, :]
        else:
            print(f"Warning: Cannot extract channel {channel_idx} from data shape {data.shape}")
    
    print(f"Final data shape: {data.shape}")
    return data

# Test the loader (with limited slices to avoid memory issues)
test_tiff_path = Path('1msec_worm/I/beads_alpha_worm2/beads_alpha_worm2_MMStack_Pos0.ome.tif')
if test_tiff_path.exists():
    print("Testing OME-TIFF loader...")
    test_meta = parse_metadata(test_tiff_path.parent / test_tiff_path.name.replace('.ome.tif', '_metadata.txt'))
    # Load only first 5 slices and first channel for testing
    test_data = load_ome_tiff(test_tiff_path, metadata=test_meta, channel_idx=0, max_slices=5)
    print(f"Test load successful! Data shape: {test_data.shape}")
    print(f"Data range: [{test_data.min()}, {test_data.max()}]")
else:
    print("Test TIFF file not found. Loader function is ready to use.")


Testing OME-TIFF loader...
Loading OME-TIFF: 1msec_worm/I/beads_alpha_worm2/beads_alpha_worm2_MMStack_Pos0.ome.tif
Raw data shape: (200, 2, 2304, 2304)
Data dtype: uint16
Limited to 5 slices
Final data shape: (5, 2304, 2304)
Test load successful! Data shape: (5, 2304, 2304)
Data range: [35, 195]


### Important Notes on Image Display and Data Preservation

**Image Quality and Dynamic Range:**
- The display functions now preserve the **full dynamic range** of your data
- Images are scaled linearly from their actual min/max values to [0,1] for display
- **No clipping or percentile-based normalization** is applied by default
- The actual pixel values are preserved in memory - only the display is scaled

**Data Formats:**
- Currently using NumPy arrays loaded from OME-TIFF files
- For very large datasets, consider using **Zarr** or **HDF5** formats for:
  - Memory-efficient chunked access
  - Lazy loading of slices
  - Better performance with large stacks
  - Example: `zarr.open('data.zarr', mode='r')` for chunked array access

**Memory Considerations:**
- Full stacks are now loaded by default (MAX_SLICES_FOR_DISPLAY = None)
- For 200 slices × 2304×2304 × 2 channels × 2 bytes = ~4.2 GB per arm
- If memory is limited, set MAX_SLICES_FOR_DISPLAY to a smaller number
- Consider using memory mapping or chunked loading for very large datasets


In [5]:
def discover_acquisitions(root_dir='.'):
    """
    Discover all alpha/beta acquisition pairs in the directory structure.
    
    Parameters:
    -----------
    root_dir : str or Path
        Root directory to search for acquisitions
        
    Returns:
    --------
    list : List of dictionaries, each containing:
        - 'condition': Top-level folder name (e.g., '1msec_worm')
        - 'run': Second-level folder name (e.g., 'I')
        - 'alpha_path': Path to alpha folder
        - 'beta_path': Path to beta folder
        - 'alpha_metadata': Path to alpha metadata file
        - 'beta_metadata': Path to beta metadata file
        - 'alpha_tiff': Path to alpha OME-TIFF file
        - 'beta_tiff': Path to beta OME-TIFF file
    """
    root_path = Path(root_dir)
    acquisitions = []
    
    # Find all top-level directories (acquisition conditions)
    for condition_dir in sorted(root_path.iterdir()):
        if not condition_dir.is_dir():
            continue
        
        condition_name = condition_dir.name
        
        # Find all second-level directories (runs)
        for run_dir in sorted(condition_dir.iterdir()):
            if not run_dir.is_dir():
                continue
            
            run_name = run_dir.name
            
            # Look for alpha and beta folders
            alpha_folder = None
            beta_folder = None
            
            for subfolder in run_dir.iterdir():
                if not subfolder.is_dir():
                    continue
                
                folder_name = subfolder.name.lower()
                if 'alpha' in folder_name and 'beads' in folder_name:
                    alpha_folder = subfolder
                elif 'beta' in folder_name and 'beads' in folder_name:
                    beta_folder = subfolder
            
            # If we found both alpha and beta, create an acquisition entry
            if alpha_folder and beta_folder:
                # Find metadata and TIFF files
                alpha_metadata = None
                alpha_tiff = None
                beta_metadata = None
                beta_tiff = None
                
                # Look for metadata and TIFF files in alpha folder
                for file in alpha_folder.iterdir():
                    if file.suffix == '.txt' and 'metadata' in file.name:
                        alpha_metadata = file
                    elif file.suffix == '.tif' or file.suffix == '.tiff':
                        alpha_tiff = file
                
                # Look for metadata and TIFF files in beta folder
                for file in beta_folder.iterdir():
                    if file.suffix == '.txt' and 'metadata' in file.name:
                        beta_metadata = file
                    elif file.suffix == '.tif' or file.suffix == '.tiff':
                        beta_tiff = file
                
                # Only add if we have all required files
                if alpha_metadata and alpha_tiff and beta_metadata and beta_tiff:
                    acquisitions.append({
                        'condition': condition_name,
                        'run': run_name,
                        'alpha_path': alpha_folder,
                        'beta_path': beta_folder,
                        'alpha_metadata': alpha_metadata,
                        'beta_metadata': beta_metadata,
                        'alpha_tiff': alpha_tiff,
                        'beta_tiff': beta_tiff
                    })
    
    return acquisitions

# Discover all acquisitions
acquisitions = discover_acquisitions('.')
print(f"Found {len(acquisitions)} acquisition pairs:")
for i, acq in enumerate(acquisitions):
    print(f"  {i+1}. {acq['condition']}/{acq['run']}: alpha={acq['alpha_path'].name}, beta={acq['beta_path'].name}")


Found 8 acquisition pairs:
  1. 10msec_worm/I: alpha=beads_alpha_worm2_4, beta=beads_beta_worm2_4
  2. 1msec_worm/I: alpha=beads_alpha_worm2, beta=beads_beta_worm2
  3. 1msec_worm/II: alpha=beads_alpha_worm2_1_1, beta=beads_beta_worm2_1_1
  4. 200msec_worm/I: alpha=beads_alpha_worm2, beta=beads_beta_worm2
  5. 200msec_worm/II: alpha=beads_alpha_worm2_1, beta=beads_beta_worm2_1
  6. 200msec_worm/III: alpha=beads_alpha_worm2_2, beta=beads_beta_worm2_2
  7. 200msec_worm/IV: alpha=beads_alpha_worm2_3, beta=beads_beta_worm2_3
  8. 20msec_worm/I: alpha=beads_alpha_worm2_2_1, beta=beads_beta_worm2_2_1


## Section 5: Temporal Alignment

The alpha and beta arms may have slight timing differences. We need to:
1. Extract start times from metadata
2. Account for the delay between sides (`delayBeforeSide`)
3. Calculate frame-by-frame timing
4. Align the sequences temporally

This is important for creating synchronized side-by-side videos.


In [6]:
def parse_start_time(time_str):
    """
    Parse the StartTime string from metadata into a datetime object.
    
    Format: "2025-11-12 17:04:10 -0500"
    """
    try:
        # Try parsing with timezone
        dt = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S %z")
        return dt
    except:
        try:
            # Try without timezone
            dt = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")
            return dt
        except:
            print(f"Warning: Could not parse time string: {time_str}")
            return None

def calculate_temporal_alignment(alpha_meta, beta_meta):
    """
    Calculate temporal alignment between alpha and beta acquisitions.
    
    Parameters:
    -----------
    alpha_meta : dict
        Parsed metadata for alpha arm
    beta_meta : dict
        Parsed metadata for beta arm
        
    Returns:
    --------
    dict : Dictionary containing:
        - 'time_offset_sec': Time difference between start times (beta - alpha)
        - 'alpha_start': Alpha start datetime
        - 'beta_start': Beta start datetime
        - 'slice_period_ms': Average slice period
        - 'frame_times_alpha': Array of frame times for alpha (relative to alpha start)
        - 'frame_times_beta': Array of frame times for beta (relative to beta start)
    """
    alpha_start = parse_start_time(alpha_meta['start_time'])
    beta_start = parse_start_time(beta_meta['start_time'])
    
    if alpha_start is None or beta_start is None:
        print("Warning: Could not parse start times. Using default alignment.")
        time_offset_sec = 0.0
    else:
        # Calculate time difference (beta - alpha) in seconds
        time_diff = beta_start - alpha_start
        time_offset_sec = time_diff.total_seconds()
    
    # Get slice periods
    alpha_slice_period = alpha_meta['slice_period_ms'] / 1000.0  # Convert to seconds
    beta_slice_period = beta_meta['slice_period_ms'] / 1000.0
    
    # Use average slice period
    avg_slice_period = (alpha_slice_period + beta_slice_period) / 2.0
    
    # Account for delay before side
    delay_before_side = alpha_meta.get('delay_before_side', 0.25)
    
    # Calculate frame times
    num_slices = min(alpha_meta['slices'], beta_meta['slices'])
    
    # Alpha frame times: starts immediately
    frame_times_alpha = np.arange(num_slices) * alpha_slice_period
    
    # Beta frame times: starts after delay_before_side
    frame_times_beta = np.arange(num_slices) * beta_slice_period + delay_before_side
    
    return {
        'time_offset_sec': time_offset_sec,
        'alpha_start': alpha_start,
        'beta_start': beta_start,
        'slice_period_ms': avg_slice_period * 1000,
        'delay_before_side': delay_before_side,
        'frame_times_alpha': frame_times_alpha,
        'frame_times_beta': frame_times_beta,
        'num_slices': num_slices
    }

# Test temporal alignment with first acquisition
if len(acquisitions) > 0:
    acq = acquisitions[0]
    alpha_meta = parse_metadata(acq['alpha_metadata'])
    beta_meta = parse_metadata(acq['beta_metadata'])
    
    alignment = calculate_temporal_alignment(alpha_meta, beta_meta)
    print("Temporal alignment calculated:")
    print(f"  Alpha start: {alignment['alpha_start']}")
    print(f"  Beta start: {alignment['beta_start']}")
    print(f"  Time offset: {alignment['time_offset_sec']:.3f} seconds")
    print(f"  Delay before side: {alignment['delay_before_side']:.3f} seconds")
    print(f"  Average slice period: {alignment['slice_period_ms']:.2f} ms")
    print(f"  Number of slices: {alignment['num_slices']}")
else:
    print("No acquisitions found for temporal alignment test.")


Temporal alignment calculated:
  Alpha start: 2025-11-12 16:49:53-05:00
  Beta start: 2025-11-12 16:49:51-05:00
  Time offset: -2.000 seconds
  Delay before side: 0.250 seconds
  Average slice period: 21.50 ms
  Number of slices: 200


## Section 6: Spatial Information Extraction

Extract spatial calibration and position information from metadata. This information will be useful for future alignment and registration steps.

Key spatial parameters:
- Pixel size (μm per pixel)
- Z-step size (μm between slices)
- Stage positions (X, Y)
- Rotation information (MVRotations: "0_90" suggests 90° rotation between arms)


In [7]:
def extract_spatial_info(alpha_meta, beta_meta):
    """
    Extract spatial calibration and position information.
    
    Parameters:
    -----------
    alpha_meta : dict
        Parsed metadata for alpha arm
    beta_meta : dict
        Parsed metadata for beta arm
        
    Returns:
    --------
    dict : Dictionary containing spatial information
    """
    def parse_position(pos_str):
        """Parse position string like '-0 μm' or '0.1 μm'"""
        try:
            # Extract number from string
            value = float(pos_str.split()[0])
            return value
        except:
            return 0.0
    
    spatial_info = {
        'pixel_size_um': {
            'alpha': alpha_meta['pixel_size_um'],
            'beta': beta_meta['pixel_size_um'],
            'average': (alpha_meta['pixel_size_um'] + beta_meta['pixel_size_um']) / 2.0
        },
        'z_step_um': {
            'alpha': alpha_meta['z_step_um'],
            'beta': beta_meta['z_step_um'],
            'average': (alpha_meta['z_step_um'] + beta_meta['z_step_um']) / 2.0
        },
        'position_x': {
            'alpha': parse_position(alpha_meta['position_x']),
            'beta': parse_position(beta_meta['position_x'])
        },
        'position_y': {
            'alpha': parse_position(alpha_meta['position_y']),
            'beta': parse_position(beta_meta['position_y'])
        },
        'mv_rotations': {
            'alpha': alpha_meta['mv_rotations'],
            'beta': beta_meta['mv_rotations']
        },
        'image_dimensions': {
            'width': alpha_meta['width'],
            'height': alpha_meta['height'],
            'slices': min(alpha_meta['slices'], beta_meta['slices'])
        }
    }
    
    # Calculate physical dimensions
    spatial_info['physical_dimensions_um'] = {
        'xy': {
            'width': spatial_info['image_dimensions']['width'] * spatial_info['pixel_size_um']['average'],
            'height': spatial_info['image_dimensions']['height'] * spatial_info['pixel_size_um']['average']
        },
        'z': {
            'depth': spatial_info['image_dimensions']['slices'] * spatial_info['z_step_um']['average']
        }
    }
    
    return spatial_info

# Test spatial info extraction
if len(acquisitions) > 0:
    acq = acquisitions[0]
    alpha_meta = parse_metadata(acq['alpha_metadata'])
    beta_meta = parse_metadata(acq['beta_metadata'])
    
    spatial = extract_spatial_info(alpha_meta, beta_meta)
    print("Spatial information:")
    print(f"  Pixel size: {spatial['pixel_size_um']['average']:.4f} μm/pixel")
    print(f"  Z-step: {spatial['z_step_um']['average']:.4f} μm/slice")
    print(f"  Physical volume: {spatial['physical_dimensions_um']['xy']['width']:.2f} × {spatial['physical_dimensions_um']['xy']['height']:.2f} × {spatial['physical_dimensions_um']['z']['depth']:.2f} μm")
    print(f"  Alpha position: ({spatial['position_x']['alpha']:.3f}, {spatial['position_y']['alpha']:.3f}) μm")
    print(f"  Beta position: ({spatial['position_x']['beta']:.3f}, {spatial['position_y']['beta']:.3f}) μm")
    print(f"  MVRotations - Alpha: {spatial['mv_rotations']['alpha']}, Beta: {spatial['mv_rotations']['beta']}")
    if '90' in spatial['mv_rotations']['alpha']:
        print("  Note: 90° rotation detected between arms (typical for diSPIM)")
else:
    print("No acquisitions found for spatial info test.")


Spatial information:
  Pixel size: 0.1220 μm/pixel
  Z-step: 0.7000 μm/slice
  Physical volume: 281.09 × 281.09 × 140.00 μm
  Alpha position: (-0.000, -0.000) μm
  Beta position: (-0.000, 0.100) μm
  MVRotations - Alpha: 0_90, Beta: 0_90
  Note: 90° rotation detected between arms (typical for diSPIM)


## Section 7: Video Creation Pipeline

Create functions to generate side-by-side video sequences from alpha and beta data. The videos will:
- Display alpha and beta channels side-by-side
- Use proper frame rates based on slice periods
- Normalize intensity for visualization
- Support different channel selections


In [8]:
def scale_image_for_display(img, vmin=None, vmax=None):
    """
    Scale image for display without clipping data - preserves full dynamic range.
    Uses linear scaling to 0-1 range for matplotlib display.
    
    Parameters:
    -----------
    img : numpy.ndarray
        Input image (any dtype)
    vmin : float or None
        Minimum value for scaling (None = use image min)
    vmax : float or None
        Maximum value for scaling (None = use image max)
        
    Returns:
    --------
    tuple : (scaled_image, vmin_used, vmax_used)
        scaled_image is float64 in range [0, 1] for matplotlib
    """
    if vmin is None:
        vmin = float(img.min())
    if vmax is None:
        vmax = float(img.max())
    
    # Avoid division by zero
    if vmax == vmin:
        scaled = np.zeros_like(img, dtype=np.float64)
    else:
        scaled = (img.astype(np.float64) - vmin) / (vmax - vmin)
        scaled = np.clip(scaled, 0, 1)
    
    return scaled, vmin, vmax

def create_side_by_side_frame(alpha_slice, beta_slice, normalize=False, vmin_alpha=None, vmax_alpha=None, vmin_beta=None, vmax_beta=None):
    """
    Create a side-by-side frame from alpha and beta slices.
    Returns scaled images ready for matplotlib display (preserves full dynamic range).
    
    Parameters:
    -----------
    alpha_slice : numpy.ndarray
        Single slice from alpha arm (height, width)
    beta_slice : numpy.ndarray
        Single slice from beta arm (height, width)
    normalize : bool
        DEPRECATED - kept for compatibility but not used
    vmin_alpha, vmax_alpha : float or None
        Display range for alpha (None = use full range)
    vmin_beta, vmax_beta : float or None
        Display range for beta (None = use full range)
        
    Returns:
    --------
    tuple : (side_by_side_image, vmin_alpha, vmax_alpha, vmin_beta, vmax_beta)
        Image is float64 in [0,1] range for matplotlib imshow
    """
    # Scale images for display (preserves full dynamic range)
    alpha_scaled, vmin_a, vmax_a = scale_image_for_display(alpha_slice, vmin_alpha, vmax_alpha)
    beta_scaled, vmin_b, vmax_b = scale_image_for_display(beta_slice, vmin_beta, vmax_beta)
    
    # Create side-by-side image
    side_by_side = np.hstack([alpha_scaled, beta_scaled])
    
    return side_by_side, vmin_a, vmax_a, vmin_b, vmax_b

def create_video_from_stacks(alpha_data, beta_data, output_path, 
                             channel_alpha=0, channel_beta=0,
                             fps=10, max_slices=None, normalize=True):
    """
    Create a video file from alpha and beta image stacks.
    
    Parameters:
    -----------
    alpha_data : numpy.ndarray
        Alpha image stack (slices, channels, height, width) or (slices, height, width)
    beta_data : numpy.ndarray
        Beta image stack (slices, channels, height, width) or (slices, height, width)
    output_path : str or Path
        Output video file path
    channel_alpha : int
        Channel index to use for alpha (if multi-channel)
    channel_beta : int
        Channel index to use for beta (if multi-channel)
    fps : float
        Frames per second for output video
    max_slices : int or None
        Maximum number of slices to include (for testing)
    normalize : bool
        Whether to normalize intensities
        
    Returns:
    --------
    str : Path to created video file
    """
    # Extract channels if needed
    if len(alpha_data.shape) == 4:
        alpha_slices = alpha_data[:, channel_alpha, :, :]
    else:
        alpha_slices = alpha_data
    
    if len(beta_data.shape) == 4:
        beta_slices = beta_data[:, channel_beta, :, :]
    else:
        beta_slices = beta_data
    
    # Limit slices if requested
    num_slices = min(alpha_slices.shape[0], beta_slices.shape[0])
    if max_slices is not None:
        num_slices = min(num_slices, max_slices)
    
    alpha_slices = alpha_slices[:num_slices]
    beta_slices = beta_slices[:num_slices]
    
    print(f"Creating video with {num_slices} frames at {fps} fps...")
    
    # Create frames
    frames = []
    for i in range(num_slices):
        frame, _, _, _, _ = create_side_by_side_frame(alpha_slices[i], beta_slices[i], normalize=normalize)
        # Convert to uint8 for video (0-255 range)
        frame_uint8 = (frame * 255).astype(np.uint8)
        # Convert to RGB for video
        if len(frame_uint8.shape) == 2:
            frame_uint8 = np.stack([frame_uint8, frame_uint8, frame_uint8], axis=-1)
        frames.append(frame_uint8)
        
        if (i + 1) % 50 == 0:
            print(f"  Processed {i + 1}/{num_slices} frames...")
    
    # Save video
    print(f"Saving video to {output_path}...")
    imageio.mimwrite(str(output_path), frames, fps=fps, codec='libx264', quality=8)
    
    print(f"Video saved successfully!")
    return str(output_path)

print("Video creation functions defined!")


Video creation functions defined!


## Section 8: Interactive Visualization

Create interactive widgets for exploring the data:
- Z-slice selector slider
- Channel selector
- Playback controls
- Side-by-side display


In [ ]:
class InteractiveViewer:
    """
    Interactive viewer for diSPIM data with widgets.
    """
    def __init__(self, alpha_data, beta_data, alpha_meta=None, beta_meta=None):
        """
        Initialize the viewer with data.
        
        Parameters:
        -----------
        alpha_data : numpy.ndarray
            Alpha image stack
        beta_data : numpy.ndarray
            Beta image stack
        alpha_meta : dict, optional
            Alpha metadata
        beta_meta : dict, optional
            Beta metadata
        """
        self.alpha_data = alpha_data
        self.beta_data = beta_data
        self.alpha_meta = alpha_meta or {}
        self.beta_meta = beta_meta or {}
        
        # Determine data structure
        if len(alpha_data.shape) == 4:
            self.num_slices, self.num_channels_alpha, self.height, self.width = alpha_data.shape
            self.alpha_channels = self.alpha_meta.get('channel_names', [f'Channel {i}' for i in range(self.num_channels_alpha)])
        else:
            self.num_slices, self.height, self.width = alpha_data.shape
            self.num_channels_alpha = 1
            self.alpha_channels = ['Channel 0']
        
        if len(beta_data.shape) == 4:
            _, self.num_channels_beta, _, _ = beta_data.shape
            self.beta_channels = self.beta_meta.get('channel_names', [f'Channel {i}' for i in range(self.num_channels_beta)])
        else:
            self.num_channels_beta = 1
            self.beta_channels = ['Channel 0']
        
        # Create figure and axes
        self.fig, self.ax = plt.subplots(1, 1, figsize=(16, 8))
        self.ax.axis('off')
        
        # Initialize image object (will be created on first update)
        self.im = None
        self.text_alpha = None
        self.text_beta = None
        self.text_slice = None
        
    def update_display(self, slice_idx, channel_alpha=0, channel_beta=0):
        """
        Update the display with a specific slice and channels.
        """
        # Extract slices
        if len(self.alpha_data.shape) == 4:
            alpha_slice = self.alpha_data[slice_idx, channel_alpha, :, :]
        else:
            alpha_slice = self.alpha_data[slice_idx, :, :]
        
        if len(self.beta_data.shape) == 4:
            beta_slice = self.beta_data[slice_idx, channel_beta, :, :]
        else:
            beta_slice = self.beta_data[slice_idx, :, :]
        
        # Create side-by-side frame (preserves full dynamic range)
        frame, vmin_a, vmax_a, vmin_b, vmax_b = create_side_by_side_frame(
            alpha_slice, beta_slice, normalize=False
        )
        
        # Update or create image
        if self.im is None:
            # First time: create image
            self.im = self.ax.imshow(frame, cmap='gray', vmin=0, vmax=1, aspect='equal')
            self.ax.axis('off')
        else:
            # Update existing image data
            self.im.set_data(frame)
        
        # Update or create text labels
        alpha_range = f"[{vmin_a:.0f}, {vmax_a:.0f}]"
        beta_range = f"[{vmin_b:.0f}, {vmax_b:.0f}]"
        
        if self.text_alpha is None:
            # Create text labels
            self.text_alpha = self.ax.text(
                self.width // 2, 30,
                f'Alpha {alpha_range}',
                ha='center', va='top', color='white', fontsize=12, weight='bold',
                bbox=dict(boxstyle='round', facecolor='black', alpha=0.7)
            )
            self.text_beta = self.ax.text(
                self.width + self.width // 2, 30,
                f'Beta {beta_range}',
                ha='center', va='top', color='white', fontsize=12, weight='bold',
                bbox=dict(boxstyle='round', facecolor='black', alpha=0.7)
            )
            self.text_slice = self.ax.text(
                self.width, self.height - 30,
                f'Slice {slice_idx}/{self.num_slices-1}',
                ha='center', va='bottom', color='white', fontsize=12,
                bbox=dict(boxstyle='round', facecolor='black', alpha=0.7)
            )
        else:
            # Update existing text
            self.text_alpha.set_text(f'Alpha {alpha_range}')
            self.text_beta.set_text(f'Beta {beta_range}')
            self.text_slice.set_text(f'Slice {slice_idx}/{self.num_slices-1}')
        
        # Redraw only what's necessary
        self.fig.canvas.draw_idle()  # Use draw_idle for better widget responsiveness
    
    def create_widgets(self):
        """
        Create interactive widgets.
        """
        slice_slider = widgets.IntSlider(
            value=0,
            min=0,
            max=self.num_slices - 1,
            step=1,
            description='Slice:',
            continuous_update=True,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='500px')
        )
        
        channel_alpha_dropdown = widgets.Dropdown(
            options=list(range(self.num_channels_alpha)),
            value=0,
            description='Alpha Channel:',
            layout=widgets.Layout(width='200px')
        )
        
        channel_beta_dropdown = widgets.Dropdown(
            options=list(range(self.num_channels_beta)),
            value=0,
            description='Beta Channel:',
            layout=widgets.Layout(width='200px')
        )
        
        # Create output widget to capture display updates
        output = widgets.Output(layout={'border': '1px solid black', 'height': '600px'})
        
        # Display figure once in output
        with output:
            self.update_display(0, 0, 0)
            display(self.fig)
        
        def on_change(slice_idx, ch_alpha, ch_beta):
            # Update the display (figure is already shown, just update data)
            self.update_display(slice_idx, ch_alpha, ch_beta)
        
        # Create interactive widget
        interactive_widget = interactive(
            on_change,
            slice_idx=slice_slider,
            ch_alpha=channel_alpha_dropdown,
            ch_beta=channel_beta_dropdown
        )
        
        # Combine widgets vertically
        return widgets.VBox([
            widgets.HBox([slice_slider]),
            widgets.HBox([channel_alpha_dropdown, channel_beta_dropdown]),
            output
        ])

print("Interactive viewer class defined!")


Interactive viewer class defined!


## Section 9: Example Usage

Now let's put it all together! We'll:
1. Select an acquisition pair
2. Load the data (with optional downsampling for initial exploration)
3. Display some frames
4. Create a sample video


In [11]:
# Select an acquisition to work with
# You can change this index to select a different acquisition
ACQUISITION_INDEX = 0  # 0 = first acquisition, 1 = second, etc.

if len(acquisitions) > ACQUISITION_INDEX:
    selected_acq = acquisitions[ACQUISITION_INDEX]
    print(f"Selected acquisition: {selected_acq['condition']}/{selected_acq['run']}")
    print(f"  Alpha: {selected_acq['alpha_path'].name}")
    print(f"  Beta: {selected_acq['beta_path'].name}")
else:
    print(f"Error: Acquisition index {ACQUISITION_INDEX} not available. Found {len(acquisitions)} acquisitions.")
    selected_acq = None


Selected acquisition: 10msec_worm/I
  Alpha: beads_alpha_worm2_4
  Beta: beads_beta_worm2_4


In [12]:
# Load metadata for selected acquisition
if selected_acq:
    print("Loading metadata...")
    alpha_meta = parse_metadata(selected_acq['alpha_metadata'])
    beta_meta = parse_metadata(selected_acq['beta_metadata'])
    
    print("\nAlpha metadata summary:")
    print(f"  Dimensions: {alpha_meta['width']}×{alpha_meta['height']}×{alpha_meta['slices']}")
    print(f"  Channels: {alpha_meta['channels']} ({', '.join(alpha_meta['channel_names'])})")
    print(f"  Start time: {alpha_meta['start_time']}")
    
    print("\nBeta metadata summary:")
    print(f"  Dimensions: {beta_meta['width']}×{beta_meta['height']}×{beta_meta['slices']}")
    print(f"  Channels: {beta_meta['channels']} ({', '.join(beta_meta['channel_names'])})")
    print(f"  Start time: {beta_meta['start_time']}")
    
    # Calculate temporal alignment
    alignment = calculate_temporal_alignment(alpha_meta, beta_meta)
    print(f"\nTemporal alignment:")
    print(f"  Time offset: {alignment['time_offset_sec']:.3f} seconds")
    print(f"  Average slice period: {alignment['slice_period_ms']:.2f} ms")
    
    # Extract spatial info
    spatial = extract_spatial_info(alpha_meta, beta_meta)
    print(f"\nSpatial information:")
    print(f"  Pixel size: {spatial['pixel_size_um']['average']:.4f} μm/pixel")
    print(f"  Z-step: {spatial['z_step_um']['average']:.4f} μm/slice")
    print(f"  Volume size: {spatial['physical_dimensions_um']['xy']['width']:.2f} × {spatial['physical_dimensions_um']['xy']['height']:.2f} × {spatial['physical_dimensions_um']['z']['depth']:.2f} μm")
else:
    print("No acquisition selected.")


Loading metadata...

Alpha metadata summary:
  Dimensions: 2304×2304×200
  Channels: 2 (HamCam2, HamCam1)
  Start time: 2025-11-12 16:49:53 -0500

Beta metadata summary:
  Dimensions: 2304×2304×200
  Channels: 2 (HamuHam4, HamuHam3)
  Start time: 2025-11-12 16:49:51 -0500

Temporal alignment:
  Time offset: -2.000 seconds
  Average slice period: 21.50 ms

Spatial information:
  Pixel size: 0.1220 μm/pixel
  Z-step: 0.7000 μm/slice
  Volume size: 281.09 × 281.09 × 140.00 μm


### Load Image Data

**Note**: Loading full stacks can be memory-intensive. For initial exploration, we'll load a subset of slices. 
You can adjust `MAX_SLICES_FOR_DISPLAY` to load more or fewer slices. Set to `None` to load all slices.


In [13]:
# Set maximum slices to load (None = load all, smaller number = faster loading for testing)
MAX_SLICES_FOR_DISPLAY = None  # Load full stack  # Change this to load more/fewer slices

if selected_acq:
    print("Loading OME-TIFF files...")
    print("This may take a few moments for large files...")
    
    # Load alpha data (first channel, limited slices)
    print("\nLoading alpha data...")
    alpha_data = load_ome_tiff(
        selected_acq['alpha_tiff'], 
        metadata=alpha_meta,
        channel_idx=None,  # Load all channels
        max_slices=MAX_SLICES_FOR_DISPLAY
    )
    
    # Load beta data (first channel, limited slices)
    print("\nLoading beta data...")
    beta_data = load_ome_tiff(
        selected_acq['beta_tiff'],
        metadata=beta_meta,
        channel_idx=None,  # Load all channels
        max_slices=MAX_SLICES_FOR_DISPLAY
    )
    
    print(f"\nData loaded successfully!")
    print(f"Alpha data shape: {alpha_data.shape}")
    print(f"Beta data shape: {beta_data.shape}")
    print(f"Alpha data dtype: {alpha_data.dtype}, range: [{alpha_data.min()}, {alpha_data.max()}]")
    print(f"Beta data dtype: {beta_data.dtype}, range: [{beta_data.min()}, {beta_data.max()}]")
else:
    print("No acquisition selected.")
    alpha_data = None
    beta_data = None


Loading OME-TIFF files...
This may take a few moments for large files...

Loading alpha data...
Loading OME-TIFF: 10msec_worm/I/beads_alpha_worm2_4/beads_alpha_worm2_MMStack_Pos0.ome.tif
Raw data shape: (200, 2, 2304, 2304)
Data dtype: uint16
Final data shape: (200, 2, 2304, 2304)

Loading beta data...
Loading OME-TIFF: 10msec_worm/I/beads_beta_worm2_4/beads_beta_worm2_MMStack_Pos0.ome.tif
Raw data shape: (200, 2, 2304, 2304)
Data dtype: uint16
Final data shape: (200, 2, 2304, 2304)

Data loaded successfully!
Alpha data shape: (200, 2, 2304, 2304)
Beta data shape: (200, 2, 2304, 2304)
Alpha data dtype: uint16, range: [17, 12712]
Beta data dtype: uint16, range: [0, 11448]


### Display Sample Frames

Let's visualize a few slices side-by-side to get a sense of the data.


In [14]:
if alpha_data is not None and beta_data is not None:
    # Display a few sample slices
    num_samples = min(5, alpha_data.shape[0])
    sample_indices = np.linspace(0, alpha_data.shape[0] - 1, num_samples, dtype=int)
    
    fig, axes = plt.subplots(1, num_samples, figsize=(20, 4))
    if num_samples == 1:
        axes = [axes]
    
    for i, slice_idx in enumerate(sample_indices):
        # Extract slices (first channel)
        if len(alpha_data.shape) == 4:
            alpha_slice = alpha_data[slice_idx, 0, :, :]
            beta_slice = beta_data[slice_idx, 0, :, :]
        else:
            alpha_slice = alpha_data[slice_idx, :, :]
            beta_slice = beta_data[slice_idx, :, :]
        
        # Create side-by-side frame
        frame, _, _, _, _ = create_side_by_side_frame(alpha_slice, beta_slice, normalize=False)
        
        axes[i].imshow(frame, cmap='gray', vmin=0, vmax=1)
        axes[i].axis('off')
        axes[i].set_title(f'Slice {slice_idx}', fontsize=10)
    
    plt.suptitle(f'Sample slices from {selected_acq["condition"]}/{selected_acq["run"]}', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("No data loaded to display.")


<IPython.core.display.Javascript object>

### Interactive Exploration

Use the widgets below to explore different slices and channels interactively.


In [15]:
if alpha_data is not None and beta_data is not None:
    # Create interactive viewer
    viewer = InteractiveViewer(alpha_data, beta_data, alpha_meta, beta_meta)
    
    # Create and display widgets (figure will be shown in output widget)
    interactive_widget = viewer.create_widgets()
    display(interactive_widget)
else:
    print("No data loaded for interactive viewing.")


<IPython.core.display.Javascript object>

### Create Video Output

Create a video file showing the alpha and beta channels side-by-side. The video will be saved in the current directory.


In [16]:
if alpha_data is not None and beta_data is not None:
    # Calculate appropriate frame rate based on slice period
    slice_period_ms = alignment['slice_period_ms']
    fps = 1000.0 / slice_period_ms  # frames per second
    # Cap at reasonable frame rate for video playback
    fps = min(fps, 30.0)
    
    print(f"Creating video at {fps:.2f} fps (based on {slice_period_ms:.2f} ms slice period)...")
    
    # Create output filename
    output_filename = f"{selected_acq['condition']}_{selected_acq['run']}_alpha_beta_video.mp4"
    
    # Create video (using first channel for both arms)
    video_path = create_video_from_stacks(
        alpha_data, 
        beta_data,
        output_filename,
        channel_alpha=0,
        channel_beta=0,
        fps=fps,
        max_slices=None,  # Use all loaded slices
        normalize=True
    )
    
    print(f"\nVideo saved to: {video_path}")
    print(f"You can play this video with any standard video player.")
else:
    print("No data loaded to create video.")


Creating video at 30.00 fps (based on 21.50 ms slice period)...
Creating video with 200 frames at 30.0 fps...
  Processed 50/200 frames...
  Processed 100/200 frames...
  Processed 150/200 frames...
  Processed 200/200 frames...
Saving video to 10msec_worm_I_alpha_beta_video.mp4...
Video saved successfully!

Video saved to: 10msec_worm_I_alpha_beta_video.mp4
You can play this video with any standard video player.


## Summary and Next Steps

### What We've Accomplished

1. **Metadata Parsing**: Successfully extracted acquisition parameters, dimensions, timing, and spatial information
2. **Data Loading**: Loaded OME-TIFF files with proper handling of multi-dimensional arrays
3. **Temporal Alignment**: Calculated timing offsets and frame alignment between alpha and beta arms
4. **Spatial Information**: Extracted pixel size, z-step, and position information for future registration
5. **Visualization**: Created side-by-side displays and interactive exploration tools
6. **Video Creation**: Generated video files for easy viewing

### Key Findings

- **Data Structure**: Each arm has 2 channels (two cameras), 200 slices, 2304×2304 pixels
- **Temporal Info**: Slice periods vary by acquisition condition (1ms, 10ms, 20ms, 200ms)
- **Spatial Calibration**: ~0.244 μm/pixel in XY, 0.7 μm between slices
- **Rotation**: 90° rotation between arms (MVRotations: "0_90") - important for future registration

### Future Work

1. **Image Registration**: Use spatial information to align/register alpha and beta volumes
2. **Fusion**: Combine registered alpha and beta data for improved resolution
3. **Segmentation**: Apply ML-based segmentation tools
4. **Multi-timepoint**: Extend to handle time-lapse data (currently single timepoint)
5. **Full Stack Loading**: Load complete stacks for full-resolution analysis

### Notes

- For memory efficiency, consider loading subsets of slices or downsampling for initial exploration
- The metadata contains extensive frame-by-frame information that could be used for more detailed temporal analysis
- Spatial positions (Position_X, Position_Y) can be used to align acquisitions from different positions
